In [209]:
import pandas as pd
import numpy as np
from model.booster import GBDT_Muti
from model.dataset import DataSet
from model.configs import Configs


In [210]:
x_train = np.load('data/Re/traindataReuters.npy')
y_train = np.load('data/Re/trainlabelReuters.npy')
x_test = np.load('data/Re/testdataReuters.npy')
y_test = np.load('data/Re/testlabelReuters.npy')

In [211]:
y_train.shape

(1400, 4)

In [212]:
x_train.shape

(1400, 300)

In [213]:
dataset_name = 're'
dataset_train = DataSet(x_train, y_train)
dataset_test = DataSet(x_test, y_test)
configs = Configs('configs/configs_{}.json'.format(dataset_name))

In [214]:
gbdt = GBDT_Muti(configs)
gbdt.fit(dataset_train,dataset_test)


iter1 : valid loss=1.360000
iter1 : train loss=0.332849
iter2 : valid loss=0.372926
iter2 : train loss=0.319495
iter3 : valid loss=0.362135
iter3 : train loss=0.319322
iter4 : valid loss=0.361751
iter4 : train loss=0.319325
iter5 : valid loss=0.361792
Early stop at iter 5


In [215]:
pred_prob = gbdt.predict_set_prob(dataset_test.X)
pred_labels = gbdt.predict_set_label(dataset_test.X)

In [216]:
pred_prob_df = pd.DataFrame(pred_prob)
pred_labels_df = pd.DataFrame(pred_labels)
pred_prob_df.to_csv('./result/he_pred_prob.csv', index=False)
pred_labels_df.to_csv('./result/he_pred_label.csv', index=False)

In [217]:
pred_labels_df

,0,1,2,3
0,1,1,0,0
1,1,0,1,0
2,0,0,0,0
3,1,0,0,0
4,1,1,0,0
...,...,...,...,...
595,1,0,0,0
596,1,1,0,0
597,1,0,0,0
598,1,0,0,0


In [218]:
from sklearn.metrics import accuracy_score, f1_score, hamming_loss

In [219]:
f1_score_macro = f1_score(y_test, pred_labels_df, average='macro')
f1_score_macro

0.6091333567934722

In [220]:
f1_score_micro = f1_score(y_test, pred_labels_df, average='micro')
f1_score_micro

0.8220064724919093

In [221]:
acc = accuracy_score(y_test, pred_labels_df)
acc

0.6466666666666666

In [222]:
ham = hamming_loss(y_test, pred_labels_df)
ham

0.11458333333333333

In [223]:
record = pd.read_csv('record.csv')
record

,dataset,learn_rate,max_depth,stop_iter,f1_score_macro,f1_score_micro,acc,ham
0,he,0.5,3,12,0.369850,0.667254,0.531667,0.157500
1,he,0.1,3,49,0.398767,0.714405,0.598333,0.142917
2,he,0.3,3,23,0.380834,0.702929,0.578333,0.147917
3,he,0.8,4,10,0.418323,0.731984,0.615000,0.137917
4,he,0.8,3,10,0.422834,0.728595,0.610000,0.140000
5,re,0.1,3,95,0.623452,0.847726,0.686667,0.096250
6,he,1.0,3,5,0.441586,0.738654,0.631667,0.139167
7,re,0.5,3,23,0.646683,0.843501,0.676667,0.098333
8,re,0.8,3,10,0.640336,0.842587,0.675000,0.100417
9,re,0.8,4,17,0.681009,0.838966,0.658333,0.101250


In [224]:
record = record.append([{
    "dataset": dataset_name,
    "learn_rate": configs.learn_rate,
    "max_depth": configs.max_depth,
    "stop_iter": gbdt.stop_iter,
    "f1_score_macro":f1_score_macro,
    "f1_score_micro":f1_score_micro,
    "acc":acc,
    "ham":ham
    }])
record.to_csv('record.csv', index=False)